Test Homework3

ชื่อ_รหัส 

1.ณัชณศา_6521

2.ชัญญาภัค_6567

Import Library

In [14]:
import roboticstoolbox as rtb
import numpy as np

from spatialmath import SE3
from math import pi
from HW3_utils import *
from FRA333_HW3_6521_6567 import *

ข้อที่ 1 : Jacobian Matrix

Parameter of Robot

In [15]:
d1 = 0.0892
a2 = -0.425
a3 = -0.39243
d4 = 0.109
d5 = 0.093
d6 = 0.082

DH Parameter (Modify)

In [16]:
T3_e = SE3(a3-d6,-d5,d4) * SE3.RPY(0,-pi/2,0)

robot = rtb.DHRobot(
    [
        rtb.RevoluteMDH(d= d1 ,offset= pi), 
        rtb.RevoluteMDH(alpha= pi/2), 
        rtb.RevoluteMDH(a= -a2),
    ]
    ,tool = T3_e, name="3R robot")

print(robot)

DHRobot: 3R robot, 3 joints (RRR), dynamics, modified DH parameters
┌───────┬───────┬────────────┬────────┐
│ aⱼ₋₁  │ ⍺ⱼ₋₁  │     θⱼ     │   dⱼ   │
├───────┼───────┼────────────┼────────┤
│   0.0 │  0.0° │  q1 + 180° │ 0.0892 │
│   0.0 │ 90.0° │         q2 │    0.0 │
│ 0.425 │  0.0° │         q3 │    0.0 │
└───────┴───────┴────────────┴────────┘

┌──────┬─────────────────────────────────────────────────┐
│ tool │ t = -0.47, -0.093, 0.11; rpy/xyz = 0°, -90°, 0° │
└──────┴─────────────────────────────────────────────────┘



In [17]:
print(robot.jacob0)

<bound method DHRobot.jacob0 of DHRobot: 3R robot, 3 joints (RRR), dynamics, modified DH parameters
┌───────┬───────┬────────────┬────────┐
│ aⱼ₋₁  │ ⍺ⱼ₋₁  │     θⱼ     │   dⱼ   │
├───────┼───────┼────────────┼────────┤
│   0.0 │  0.0° │  q1 + 180° │ 0.0892 │
│   0.0 │ 90.0° │         q2 │    0.0 │
│ 0.425 │  0.0° │         q3 │    0.0 │
└───────┴───────┴────────────┴────────┘

┌──────┬─────────────────────────────────────────────────┐
│ tool │ t = -0.47, -0.093, 0.11; rpy/xyz = 0°, -90°, 0° │
└──────┴─────────────────────────────────────────────────┘
>


Prove

In [11]:
def numerical_jacobian(q, delta=1e-6):
    J_numerical = np.empty((6, len(q)))

    # คำนวณตำแหน่งและการหมุนของ end-effector ที่จุดเริ่มต้น
    R, P, R_e, p_e_initial = FKHW3(q)

    for i in range(len(q)):
        # สร้าง perturbation (การเปลี่ยนแปลงเล็กๆ) ให้กับ q[i]
        q_perturbed = np.copy(q)
        q_perturbed[i] += delta

        # คำนวณตำแหน่งและการหมุนของ end-effector หลังจาก perturbation
        R, P, R_e, p_e_perturbed = FKHW3(q_perturbed)

        # คำนวณการเปลี่ยนแปลงของตำแหน่งและการหมุน
        dp = (p_e_perturbed - p_e_initial) / delta

        # ใช้เวกเตอร์ต่างเชิงอนุพันธ์เพื่อคำนวณ angular velocity ด้วยวิธี numerical
        J_numerical[:3, i] = dp  # Linear part
        # Angular part: ยังไม่ได้ใช้ส่วนนี้ (ขึ้นกับทิศทางการหมุนที่ได้จาก p_e)

    return J_numerical

# สุ่มค่า q เพื่อทดสอบ
q_random = np.random.rand(3) * 2 * np.pi  # ค่า q สุ่มในช่วง [0, 2π]

# คำนวณ Jacobian ด้วยฟังก์ชันที่เขียน
J_analytic = endEffectorJacobianHW3(q_random)

# คำนวณ Jacobian ด้วยวิธี numerical differentiation
J_numerical = numerical_jacobian(q_random)

# แสดงผลลัพธ์
print("Jacobian ที่คำนวณได้จากฟังก์ชันหุ่นยนต์ (analytic):\n", J_analytic)
print("\nJacobian ที่คำนวณได้จาก Numerical Differentiation:\n", J_numerical)

# ตรวจสอบว่า Jacobian ที่คำนวณได้นั้นใกล้เคียงกัน
difference = np.abs(J_analytic - J_numerical)
print("\nความแตกต่างระหว่างสองวิธี:\n", difference)

Jacobian ที่คำนวณได้จากฟังก์ชันหุ่นยนต์ (analytic):
 [[ 2.06209339e-01  1.92955868e-01 -2.12467555e-01]
 [ 3.29933099e-01 -5.43690327e-02  5.98668264e-02]
 [-0.00000000e+00  3.73493152e-01  4.30123604e-01]
 [ 0.00000000e+00 -2.71208704e-01 -2.71208704e-01]
 [ 0.00000000e+00 -9.62520540e-01 -9.62520540e-01]
 [ 1.00000000e+00  6.12323426e-17  6.12323426e-17]]

Jacobian ที่คำนวณได้จาก Numerical Differentiation:
 [[ 2.06209174e-01  1.92955693e-01 -2.12467767e-01]
 [ 3.29933202e-01 -5.43689827e-02  5.98668854e-02]
 [ 0.00000000e+00  3.73493261e-01  4.30123504e-01]
 [ 0.00000000e+00 -9.97692883e-01 -9.97692883e-01]
 [ 0.00000000e+00 -6.78891689e-02 -6.78891689e-02]
 [ 1.00000000e+00  6.12323426e-17  6.12323426e-17]]

ความแตกต่างระหว่างสองวิธี:
 [[1.64905017e-07 1.74825341e-07 2.12512827e-07]
 [1.03138316e-07 5.00360750e-08 5.89921374e-08]
 [0.00000000e+00 1.09040005e-07 1.00172657e-07]
 [0.00000000e+00 7.26484179e-01 7.26484179e-01]
 [0.00000000e+00 8.94631371e-01 8.94631371e-01]
 [0.0000000

In [33]:
def proofJacobian(q: list[float], robot: rtb.DHRobot) -> bool:
    # คำนวณ Jacobian จากฟังก์ชัน endEffectorJacobianHW3
    J_e = endEffectorJacobianHW3(q)  # Jacobian ที่ได้จากฟังก์ชันที่คุณเขียน

    # คำนวณ Jacobian จาก Robotics Toolbox (อ้างอิงเฟรมเดียวกัน: base frame)
    J_ertb = robot.jacob0(q)  # Jacobian อ้างอิงจาก base frame

    allow_error = 0.0001  # ค่าความคลาดเคลื่อนที่ยอมรับได้ในการเปรียบเทียบ
    print(q)
    print("-----------Jacobian จากโค้ดของคุณ-----------")
    print(J_e)
    print("-----------Jacob0 จาก Robotics Toolbox------------")
    print(J_ertb)
    print("------------Jacobian ถูกหรือไม่------------")
    
    # เปรียบเทียบ Jacobian ทั้งสอง
    return np.allclose(J_e, J_ertb, atol=allow_error)

# สร้างค่า q แบบสุ่ม
q = np.random.rand(3) * 2 * np.pi
# เรียกใช้หุ่นยนต์ที่สร้างไว้แล้ว (เช่นหุ่นยนต์ 3 ข้อต่อ)
result = proofJacobian(q, robot)

print("Jacobian is correct:", result)

[4.16530338 5.50252146 1.00953679]
-----------Jacobian จากโค้ดของคุณ-----------
[[ 6.91169658e-01 -5.24782184e-02  1.03110745e-01]
 [-2.93365075e-01 -8.61566336e-02  1.69283084e-01]
 [ 0.00000000e+00 -7.42898075e-01 -4.40958310e-01]
 [ 0.00000000e+00  8.54044199e-01  8.54044199e-01]
 [ 0.00000000e+00 -5.20200431e-01 -5.20200431e-01]
 [ 1.00000000e+00  6.12323426e-17  6.12323426e-17]]
-----------Jacob0 จาก Robotics Toolbox------------
[[ 1.75429832e-01  2.58699701e-01  1.03110742e-01]
 [ 2.07733133e-02  4.24722810e-01  1.69283087e-01]
 [ 3.46944695e-18 -1.39018547e-01 -4.40958316e-01]
 [ 0.00000000e+00  8.54044217e-01  8.54044217e-01]
 [-5.55111512e-17 -5.20200419e-01 -5.20200419e-01]
 [ 1.00000000e+00  6.12323400e-17  6.12323400e-17]]
------------Jacobian ถูกหรือไม่------------
Jacobian is correct: False
